# 1 - BiLSTM pour Named Entity Recognition

## Introduction

Dans ce notebook, nous allons implémenter un modèle LSTM multi-couche et bidirectionnel pour faire de la reconnaissance d'entités nommés (ou NER) en utilisant le dataset CONLL2003.

## Préparer les données

On importe tout d'abord les librairies utiles.

In [49]:
import torch
import torch.nn as nn
import torch.optim as optim

from torchtext import data
from torchtext import datasets

import spacy
import numpy as np
import pandas as pd

import time
import random
import string
from itertools import chain

Les données sont téléchargeable [ici](https://github.com/davidsbatista/NER-datasets/tree/master/CONLL2003) au format .txt et sont sous la forme : 

```
-DOCSTART- -X- -X- O

SOCCER NN B-NP O
- : O O
JAPAN NNP B-NP B-LOC
GET VB B-VP O
LUCKY NNP B-NP O
WIN NNP I-NP O
, , O O
CHINA NNP B-NP B-PER
IN IN B-PP O
SURPRISE DT B-NP O
DEFEAT NN I-NP O
. . O O

Nadim NNP B-NP B-PER
Ladki NNP I-NP I-PER

AL-AIN NNP B-NP B-LOC
, , O O
United NNP B-NP B-LOC
Arab NNP I-NP I-LOC
Emirates NNPS I-NP I-LOC
1996-12-06 CD I-NP O
```

où les tags qui nous intéressent pour le named entity recognition sont ceux de la dernière colonne de chaque ligne. Par exemple, 0, B-LOC, B-PER ...

Nous allons arranger les données pour avoir une phrase par ligne et les convertir au format csv, pour qu'on puisse utiliser TorchText.

In [50]:
def process_doc(filename):
    data_test = []
    tag_test = []
    t = []
    sent = []
    label = []
    vocab = {}
    f1  = open(filename, "r") 
    for i, line in enumerate(f1):  
        if line.split(): #on ne prend pas en compte les listes vides
            vocab[line.split()[0]] = i
            sent.append(line.split()[0])
            tag = [s for s in line.split()[1:] if s not in string.punctuation]
            tag_test.append(tag[-1])
            t.append(tag[-1])

        else:
            if tag_test:
                label.append(" ".join(tag_test))
                tag_test = []
            if sent:
                data_test.append(" ".join(sent))
                sent = []
             
    return data_test, label

word_train, tag_train = process_doc('data_ner/train.txt')
word_val, tag_val = process_doc('data_ner/valid.txt')
word_test, tag_test = process_doc('data_ner/test.txt')


train = pd.DataFrame({'text':word_train, 'tag': tag_train})
validation = pd.DataFrame({'text':word_val, 'tag': tag_val})
test = pd.DataFrame({'text':word_test, 'tag': tag_test})

Nos données sont maintenant sous cette forme : 

In [51]:
train.head(7)

,text,tag
0,-DOCSTART-,O
1,EU rejects German call to boycott British lamb .,B-ORG O B-MISC O O O B-MISC O O
2,Peter Blackburn,B-PER I-PER
3,BRUSSELS 1996-08-22,B-LOC O
4,The European Commission said on Thursday it di...,O B-ORG I-ORG O O O O O O B-MISC O O O O O B-M...
5,Germany 's representative to the European Unio...,B-LOC O O O O B-ORG I-ORG O O O B-PER I-PER O ...
6,""" We do n't support any such recommendation be...",O O O O O O O O O O O O O O O O O O O O B-ORG ...


On les convertis en csv.

In [52]:
train.to_csv (r'data_ner/train.csv', index = False, header=True)
test.to_csv (r'data_ner/test.csv', index = False, header=True)
validation.to_csv (r'data_ner/valid.csv', index = False, header=True)

On a alors des données en forme pour pouvoir utiliser les méthodes de TorchText.

In [53]:
# pour la reproductibilité
SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

On utilise la méthode `Field` de TorchText pour pré-traiter nos données. On utilise seulement `lower = True` pour mettre en minuscule le texte.

Pour les tags, on définit également un `Field`.

In [54]:
TEXT = data.Field(lower = True) 
TAG = data.Field(unk_token = None) # les tags sont tous connus on a alors unk_token = None

In [55]:
train_data, valid_data, test_data = data.TabularDataset.splits(
        path="./data_ner/",
        train="train.csv",
        validation="valid.csv",
        test="test.csv", format='csv', skip_header=True,
        fields=(("text", TEXT), ("tag", TAG))
    )

On affiche le nombre de phrases dans chaque dataset.

In [56]:
print(f"Number of training examples: {len(train_data)}")
print(f"Number of validation examples: {len(valid_data)}")
print(f"Number of testing examples: {len(test_data)}")

Number of training examples: 14986
Number of validation examples: 3465
Number of testing examples: 3683


Affichons un exemple.

In [57]:
print(vars(train_data.examples[1]))

{'text': ['eu', 'rejects', 'german', 'call', 'to', 'boycott', 'british', 'lamb', '.'], 'tag': ['B-ORG', 'O', 'B-MISC', 'O', 'O', 'O', 'B-MISC', 'O', 'O']}


Ensuite, nous construisons le vocabulaire.

On importe les embeddings pré-entrainés de [GloVe](https://nlp.stanford.edu/projects/glove/).  


`unk_init` est utilisé pour initialiser les embeddings qui ne sont pas dans le vocabulaire des embeddings pré-entraîné, on les initialise en utilisant une distribution Gaussienne.


In [58]:
MIN_FREQ = 2

TEXT.build_vocab(train_data, 
                 min_freq = MIN_FREQ, # les mots qui apparaissent moins que MIN_FREQ fois seront ignorés du vocabulaire
                 vectors = "glove.6B.100d",
                 unk_init = torch.Tensor.normal_)


TAG.build_vocab(train_data)

In [59]:
print(f"Unique tokens dans le vocabulaire de TEXT vocabulary: {len(TEXT.vocab)}")
print(f"Unique tokens dans le vocabulaire de TAG : {len(TAG.vocab)}")

Unique tokens dans le vocabulaire de TEXT vocabulary: 10952
Unique tokens dans le vocabulaire de TAG : 10


On peut afficher les tags les plus fréquents.

In [60]:
def tag_percentage(tag_counts):
    
    total_count = sum([count for tag, count in tag_counts])
    
    tag_counts_percentages = [(tag, count, count/total_count) for tag, count in tag_counts]
        
    return tag_counts_percentages

In [61]:
print("Tag\t\tCount\t\tPercentage\n")

for tag, count, percent in tag_percentage(TAG.vocab.freqs.most_common()):
    print(f"{tag}\t\t{count}\t\t{percent*100:4.1f}%")

Tag		Count		Percentage

O		170522		83.4%
B-LOC		7140		 3.5%
B-PER		6600		 3.2%
B-ORG		6319		 3.1%
I-PER		4528		 2.2%
I-ORG		3704		 1.8%
B-MISC		3438		 1.7%
I-LOC		1157		 0.6%
I-MISC		1155		 0.6%


On remarque que les tags ne sont pas équilibrés.

Enfin, la dernière étape de préparation des données et de créer des itérateurs.

In [62]:
BATCH_SIZE = 128

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = BATCH_SIZE,
    device = device, sort=False)

## Construire le modèle

On définit un modèle LSTM multi-couche et bi-directionnel. L'image ci-dessous illustre l'architecture de ce modèle de manière simplifiée.


![](https://github.com/bentrevett/pytorch-pos-tagging/blob/master/assets/pos-bidirectional-lstm.png?raw=1)

L'explication détaillée est décrite [ici](https://github.com/bentrevett/pytorch-pos-tagging/blob/master/1%20-%20BiLSTM%20for%20PoS%20Tagging.ipynb) ou dans [cet article](https://colah.github.io/posts/2015-08-Understanding-LSTMs/)

In [63]:
class BiLSTMNER(nn.Module):
    def __init__(self, 
                 input_dim, 
                 embedding_dim, 
                 hidden_dim, 
                 output_dim, 
                 n_layers, 
                 bidirectional, 
                 dropout, 
                 pad_idx):
        
        super().__init__()
        
        self.embedding = nn.Embedding(input_dim, embedding_dim, padding_idx = pad_idx)
        
        self.lstm = nn.LSTM(embedding_dim, 
                            hidden_dim, 
                            num_layers = n_layers, 
                            bidirectional = bidirectional,
                            dropout = dropout if n_layers > 1 else 0)
        
        self.fc = nn.Linear(hidden_dim * 2 if bidirectional else hidden_dim, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, text):

        #text = [sent len, batch size]
        
        embedded = self.dropout(self.embedding(text)) 
        #embedded = [sent len, batch size, emb dim]
        
        outputs, (hidden, cell) = self.lstm(embedded)
        #output = [sent len, batch size, hid dim * n directions]
        #hidden/cell = [n layers * n directions, batch size, hid dim]
   
        predictions = self.fc(self.dropout(outputs))
        #predictions = [sent len, batch size, output dim]
        
        return predictions

## Entraînement

In [64]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 100 # doit être le même que la dimension du GloVe embeddings
HIDDEN_DIM = 128
OUTPUT_DIM = len(TAG.vocab)
N_LAYERS = 2
BIDIRECTIONAL = True
DROPOUT = 0.25
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

model = BiLSTMNER(INPUT_DIM, 
                        EMBEDDING_DIM, 
                        HIDDEN_DIM, 
                        OUTPUT_DIM, 
                        N_LAYERS, 
                        BIDIRECTIONAL, 
                        DROPOUT, 
                        PAD_IDX)

On initialise les poids avec une disribution Gaussienne.

In [65]:
def init_weights(m):
    for name, param in m.named_parameters():
        nn.init.normal_(param.data, mean = 0, std = 0.1)
        
model.apply(init_weights)

BiLSTMNER(
  (embedding): Embedding(10952, 100, padding_idx=1)
  (lstm): LSTM(100, 128, num_layers=2, dropout=0.25, bidirectional=True)
  (fc): Linear(in_features=256, out_features=10, bias=True)
  (dropout): Dropout(p=0.25, inplace=False)
)

On affiche combien ce modèle a de paramètres.

In [66]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'Le modèle a {count_parameters(model):,} paramètres à entraîner')

Le modèle a 1,728,554 paramètres à entraîner


On initialise la couche embedding du modèle avec les valeurs des embedding pré-entraînées importées précédemment.

In [67]:
pretrained_embeddings = TEXT.vocab.vectors
model.embedding.weight.data.copy_(pretrained_embeddings)

tensor([[-0.1117, -0.4966,  0.1631,  ...,  1.2647, -0.2753, -0.1325],
        [-0.8555, -0.7208,  1.3755,  ...,  0.0825, -1.1314,  0.3997],
        [-0.0382, -0.2449,  0.7281,  ..., -0.1459,  0.8278,  0.2706],
        ...,
        [-0.3795, -0.3950,  1.4899,  ...,  0.8631,  1.3831,  1.0091],
        [ 0.2933, -0.5548, -0.1047,  ...,  0.3955, -0.1746, -0.1650],
        [ 0.9505,  0.0562, -1.0698,  ..., -0.1683,  0.4845, -0.0838]])

On initialise également les embedding des tokens pad à zéro.

In [68]:
model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)

On définit ensuite l'optimiseur. On utilise Adam.

In [69]:
optimizer = optim.Adam(model.parameters())

La fonction de coût est la cross-entropy. Dans le vocabulaire de TAG, on a des tokens `<pad>` car dans un batch les phrases doivent avoir la même taille. En revanche, nous ne voulons par calculer la perte quand le tag vaut  `<pad>`, c'est pour cela qu'on doit les ignorer avec l'argument `ignore_index`.

In [70]:
TAG_PAD_IDX = TAG.vocab.stoi[TAG.pad_token]
criterion = nn.CrossEntropyLoss(ignore_index = TAG_PAD_IDX)

On place notre modèle et la fonction de coût sur GPU.

In [71]:
model = model.to(device)
criterion = criterion.to(device)

La fonction suivante calcule l'accuracy en ne prenant pas en compte les tokens `<pad>`.

In [115]:
def categorical_accuracy(preds, y, tag_pad_idx):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """
    max_preds = preds.argmax(dim = 1, keepdim = True) # get the index of the max probability
    non_pad_elements = (y != tag_pad_idx).nonzero()
    correct = max_preds[non_pad_elements].squeeze(1).eq(y[non_pad_elements])
    return correct.sum() / torch.FloatTensor([y[non_pad_elements].shape[0]])

def f1_loss(preds, y, tag_pad_idx):
    '''
    Retourne le score F1
    '''  
     
    max_preds = preds.argmax(dim = 1, keepdim = True) 
    non_pad_elements = (y != tag_pad_idx).nonzero()
    y_pred = (max_preds[non_pad_elements])
    y_true = (y_pred == y[non_pad_elements]).float() 
            
    tp = (y_true * y_pred).sum().float()
    tn = ((1 - y_true) * (1 - y_pred)).sum().float()
    fp = ((1 - y_true) * y_pred).sum().float()
    fn = (y_true * (1 - y_pred)).sum().float()
    
    recall = tp / (tp + fn)
    precision = tp / (tp + fp)
    
    if (tp + fn) == 0 or (tp + fp) == 0 or (recall + precision == 0):
        f1 = torch.zeros(1)
    else:
        f1 = 2* (precision*recall) / (precision + recall)
    
    return f1


Vient ensuite la fonction qui gère l'entraînement de notre modèle.

Nous avons d'abord mis le modèle en mode `train` pour activer le dropout / batch-normarlization (si utilisé). Ensuite, nous itérons sur notre itérateur, qui renvoie un batch d'exemples.

Pour chaque batch:
- on remet à zéro les gradients sur les paramètres du dernier calcul de gradient
- insérez le batch de texte dans le modèle pour obtenir des prédictions
- comme les fonctions de perte de PyTorch ne peuvent pas gérer les prédictions en 3 dimensions, nous redimensionnons nos prédictions
- calculer la perte et la précision entre les tags prédits et les tags réels
- appeler `backward` pour calculer les gradients des paramètres w.r.t. la perte
- effectuez une «step» d'optimisation pour mettre à jour les paramètres

In [116]:
def train(model, iterator, optimizer, criterion, tag_pad_idx):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        
        text = batch.text
        tags = batch.tag
        
        optimizer.zero_grad()
        
        #text = [sent len, batch size]
        
        predictions = model(text)
        
        #predictions = [sent len, batch size, output dim]
        #tags = [sent len, batch size]
        
        predictions = predictions.view(-1, predictions.shape[-1])
        tags = tags.view(-1)
        
        #predictions = [sent len * batch size, output dim]
        #tags = [sent len * batch size]
        
        loss = criterion(predictions, tags)
                
        acc = f1_loss(predictions, tags, tag_pad_idx)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

La fonction `evaluer` est similaire à la fonction` train`, sauf avec les modifications apportées afin de ne pas mettre à jour les paramètres du modèle.

`model.eval ()` est utilisé pour mettre le modèle en mode évaluation, donc dropout / batch-norm / etc. sont désactivés.

La boucle d'itération est également enveloppée dans `torch.no_grad` pour nous assurer que nous ne calculons aucun gradient. Nous n'avons pas non plus besoin d'appeler `optimizer.zero_grad ()` et `optimizer.step ()`.

In [117]:
def evaluate(model, iterator, criterion, tag_pad_idx):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:

            text = batch.text
            tags = batch.tag
            
            predictions = model(text)
            
            predictions = predictions.view(-1, predictions.shape[-1])
            tags = tags.view(-1)
            
            loss = criterion(predictions, tags)
            
            acc = f1_loss(predictions, tags, tag_pad_idx)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

On affiche la perte et l'accuracy à chaque époque, ainsi que le temps d'exécution.

In [118]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

N_EPOCHS = 10

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion, TAG_PAD_IDX)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion, TAG_PAD_IDX)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut1-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

W torch.Size([10952, 100])
I torch.Size([45, 128])


TypeError: f1_loss() missing 1 required positional argument: 'tag_pad_idx'

Nous chargeons nos meilleurs paramètres et évaluons sur les données test. 

In [ ]:
model.load_state_dict(torch.load('tut1-model.pt'))

test_loss, test_acc = evaluate(model, test_iterator, criterion, TAG_PAD_IDX)

print(f'Test Loss: {test_loss:.3f} |  Test Acc: {test_acc*100:.2f}%')

## Inférence

Nous définissons une fonction `tag_sentence` qui va:
- mettre le modèle en mode évaluation
- tokenize la phrase avec spaCy si ce n'est pas une liste
- mettre en minuscule des tokens si le `Field` l'a fait
- numériser les tokens en utilisant le vocabulaire
- découvrir quels tokens ne sont pas dans le vocabulaire, c'est-à-dire sont des tokens `<unk>`
- convertir les tokens numérisés en un tenseur et ajouter une dimension de batch
- introduire le tenseur dans le modèle
- obtenir les prédictions sur la phrase
- convertir les prédictions en tags lisibles

En plus de renvoyer les tokens et les tags, il renvoie également les tokens qui étaient des jetons `<unk>`.

In [ ]:
def tag_sentence(model, device, sentence, text_field, tag_field):
    
    model.eval()
    
    if isinstance(sentence, str):
        nlp = spacy.load('en')
        tokens = [token.text for token in nlp(sentence)]
    else:
        tokens = [token for token in sentence]

    if text_field.lower:
        tokens = [t.lower() for t in tokens]
        
    numericalized_tokens = [text_field.vocab.stoi[t] for t in tokens]

    unk_idx = text_field.vocab.stoi[text_field.unk_token]
    
    unks = [t for t, n in zip(tokens, numericalized_tokens) if n == unk_idx]
    
    token_tensor = torch.LongTensor(numericalized_tokens)
    
    token_tensor = token_tensor.unsqueeze(-1).to(device)
         
    predictions = model(token_tensor)
    
    top_predictions = predictions.argmax(-1)
    
    predicted_tags = [tag_field.vocab.itos[t.item()] for t in top_predictions]
    
    return tokens, predicted_tags, unks

On utilise un exemple du dataset train.

In [ ]:
example_index = 1

sentence = vars(train_data.examples[example_index])['text']
actual_tags = vars(train_data.examples[example_index])['tag']

print(sentence)

On utilise la fonction `tag_sentence` pour trouver les tags.

In [ ]:
tokens, pred_tags, unks = tag_sentence(model, 
                                       device, 
                                       sentence, 
                                       TEXT, 
                                       TAG)

print(unks)

On vérifie alors si le modèle a correctement prédit les tags.

In [ ]:
print("Pred. Tag\tActual Tag\tCorrect?\tToken\n")

for token, pred_tag, actual_tag in zip(tokens, pred_tags, actual_tags):
    correct = '✔' if pred_tag == actual_tag else '✘'
    print(f"{pred_tag}\t\t{actual_tag}\t\t{correct}\t\t{token}")

Essayons avec notre propre phrase. La liste unks est nulle cela signifie que tous les mots de cette phrase sont dans le vocabulaire.

In [ ]:
sentence = 'The will deliver a speech about the conflict in North Korea tomorrow in New York with my friend Mary Kate.'

tokens, tags, unks = tag_sentence(model, 
                                  device, 
                                  sentence, 
                                  TEXT, 
                                  TAG)

print(unks)

In [ ]:
print("Pred. Tag\tToken\n")

for token, tag in zip(tokens, tags):
    print(f"{tag}\t\t{token}")